In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plot
import math
import seaborn as sns
from scipy.stats import beta

from datetime import timedelta
from datetime import datetime

import requests
import json 

import re
from statistics import mean

In [2]:
#id directories
data_dir = os.getcwd()

In [3]:
#supply data

#items_df include esentiality rank, quality, cost, demand
items_df = pd.read_excel('item_sets_user_input_df.xlsx')

orders_df = pd.read_excel('Critical_Supply_Dashboard_19 items_Aug_19.xlsx', 
                          sheet_name='DATA_Orders')
orders_df.columns = orders_df.columns.str.replace(' ', '')
items_df.columns = items_df.columns.str.replace(' ', '')
orders_df.columns = orders_df.columns.str.replace('.', '_')
orders_df = orders_df.iloc[1:]
orders_df = orders_df[orders_df['Category'].notnull()]

In [4]:
#demand
demand_available_cost_df = pd.read_excel('demand_df.xlsx')
demand_available_cost_df.columns = demand_available_cost_df.columns.str.replace(' ', '')
demand_available_cost_df.columns = demand_available_cost_df.columns.str.replace('\n', '')
demand_available_cost_df.columns = demand_available_cost_df.columns.str.replace('/', '')
demand_available_cost_df.columns = demand_available_cost_df.columns.str.replace('-', '')
demand_available_cost_df = demand_available_cost_df[['Item', 'UnitsNeededDay', 
                                                     'CurrentlyInStockCitywide', 
                                                     'RecentPrice(June)']]
items_df = items_df.merge(demand_available_cost_df, how = 'left', on = 'Item')

In [5]:
#clean orders data frame to get types

orders_df['Item'] = ''
temp = ['Disinfectant Wipes', 
        'Hand Sanitizer','Hand Soap']


for i in range(len(orders_df)):
    
    category_temp = orders_df['Category'].iloc[i]
    
    if category_temp in temp:
        orders_df['Item'].iloc[i] = category_temp
    elif category_temp in ['N-95 Filter']:
        orders_df['Item'].iloc[i] = 'Masks (N95)'   
    elif category_temp in ['Surgical Mask']:
        orders_df['Item'].iloc[i] = 'Masks (Surgical)'
    elif category_temp in ['PAWS']:
        orders_df['Item'].iloc[i] = 'Antimicrobial (PAWS) Wipes'
    elif category_temp in ['DS', 'DSS']:
        orders_df['Item'].iloc[i] = 'Disinfecting Solution'
    elif category_temp in ['IA']:
        orders_df['Item'].iloc[i] = 'Isopropyl Alcohol'
    elif ((category_temp.find('Gloves') != -1) and (orders_df['Description'].iloc[i].find('exam') != -1)):
        orders_df['Item'].iloc[i] = 'Nitrile Gloves (Public Safety)'
    elif ((category_temp.find('Gloves') != -1) and (orders_df['Description'].iloc[i].find('exam') == -1)):
        orders_df['Item'].iloc[i] = 'Nitrile Gloves (General Use)'
    elif category_temp in ['CMask']:
        orders_df['Item'].iloc[i] = 'Masks (Cloth)'
    elif category_temp in ['Therm']:
        orders_df['Item'].iloc[i] = 'Thermometer'
    elif (category_temp.find('Tyvek') != -1):
        if (category_temp.find('NH') != -1):
            orders_df['Item'].iloc[i] = 'Tyvek Suits'
        else:
            orders_df['Item'].iloc[i] = 'Tyvek Suits w/hoods'
    elif category_temp in ['Gowns']:
        orders_df['Item'].iloc[i] = 'Surgical Gowns'
    elif category_temp in ['HS Refill Bags']:
        orders_df['Item'].iloc[i] = 'Purell (1200 ml stand refill)'
    elif (category_temp.find('Toilet') != -1):
        orders_df['Item'].iloc[i] = 'Toilet Paper'
    else:
        orders_df['Item'].iloc[i] = 'NA'
        
orders_df = orders_df[orders_df['Item'] != 'NA']

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
orders_df_clean = orders_df[orders_df['EstimatedDeliveryDate'] != 'TBD']
orders_df_clean = orders_df_clean[orders_df_clean['EstimatedDeliveryDate']\
                              .notnull()]
orders_df_clean['Date'] = pd.to_datetime(orders_df_clean['Date'])
orders_df_clean['EstimatedDeliveryDate'] = \
(orders_df_clean['EstimatedDeliveryDate']).astype(str)

#to filter out other misc errors
orders_df_clean['filter'] = orders_df_clean['EstimatedDeliveryDate']\
.str.contains('-') 
orders_df_clean = orders_df_clean[orders_df_clean['filter'] == True]
orders_df_clean['EstimatedDeliveryDate'] = \
pd.to_datetime(orders_df_clean['EstimatedDeliveryDate'])

In [7]:
cap_df = orders_df[['Date', 'Item', 'Delivery1QtyReceived']]
cap_df['Date'] = pd.to_datetime(cap_df['Date'])
cap_df['Week'] = cap_df['Date'].dt.week
cap_df = cap_df[cap_df['Delivery1QtyReceived'] != '--']
cap_df['Delivery1QtyReceived'] = cap_df['Delivery1QtyReceived'].fillna(0)
cap_df = cap_df.groupby(['Item', 'Week'])['Delivery1QtyReceived'].sum().reset_index()

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform eleme

In [8]:
cap_df['Delivery1QtyReceived'] = cap_df.Delivery1QtyReceived.map(int)

In [9]:
#cap fix
cap_df = cap_df.groupby(['Item'])['Delivery1QtyReceived'].max().reset_index()
items_df = items_df.merge(cap_df, how = 'left', on = 'Item')

In [10]:
#delay time df
delay_time_df = orders_df_clean[['Item', 'Date', \
                           'EstimatedDeliveryDate']]

#calculate delay time
delay_time_df['delay_time'] = \
delay_time_df['EstimatedDeliveryDate'] - delay_time_df['Date']

#merge with items df
delay_time_df = delay_time_df.merge(items_df, 
                                    on = 'Item', 
                                    how='left')

#filter negative days
delay_time_df['delay_time_days'] = ''


for i in delay_time_df.index:
    delay_time_df['delay_time_days'].iloc[i] = \
    delay_time_df['delay_time'].iloc[i].days
    
delay_time_df = delay_time_df[delay_time_df['delay_time_days'] > 0]
delay_time_df['delay_time_days'] = \
delay_time_df['delay_time_days'].astype(int)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Item,Date,EstimatedDeliveryDate,delay_time,ItemTypeID,ItemType,ItemID,EssentialityRanking(byitemtypeid),QualityRanking(byitemlistID),UnitsNeededDay,CurrentlyInStockCitywide,RecentPrice(June),Delivery1QtyReceived,delay_time_days
0,Hand Soap,2020-03-10,2020-03-16,6 days,8.0,Hand Soap,15.0,4.0,3.0,423.222222,5348.0,4.67,3000.0,6
1,Hand Soap,2020-03-10,2020-03-16,6 days,8.0,Hand Soap,15.0,4.0,3.0,423.222222,5348.0,4.67,3000.0,6
2,Nitrile Gloves (General Use),2020-03-10,2020-03-16,6 days,2.0,Gloves,5.0,2.0,2.0,30410.000000,457647.0,0.30,20500.0,6
3,Nitrile Gloves (General Use),2020-03-10,2020-03-16,6 days,2.0,Gloves,5.0,2.0,2.0,30410.000000,457647.0,0.30,20500.0,6
4,Nitrile Gloves (General Use),2020-03-10,2020-03-16,6 days,2.0,Gloves,5.0,2.0,2.0,30410.000000,457647.0,0.30,20500.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,Nitrile Gloves (Public Safety),2020-07-23,2020-07-24,1 days,2.0,Gloves,4.0,2.0,3.0,23300.000000,764750.0,0.24,86940.0,1
182,Nitrile Gloves (Public Safety),2020-07-23,2020-07-24,1 days,2.0,Gloves,4.0,2.0,3.0,23300.000000,764750.0,0.24,86940.0,1
183,Nitrile Gloves (Public Safety),2020-07-23,2020-07-24,1 days,2.0,Gloves,4.0,2.0,3.0,23300.000000,764750.0,0.24,86940.0,1
184,Nitrile Gloves (Public Safety),2020-07-23,2020-07-24,1 days,2.0,Gloves,4.0,2.0,3.0,23300.000000,764750.0,0.24,86940.0,1


In [ ]:
alpha_list= []
beta_list = []
max_list = []
min_list = []
mean_list = []

for items in delay_time_df['Item'].unique():
    temp = delay_time_df[delay_time_df['Item'] == items]
    delay_weeks = [math.floor(days/7) for days in temp['delay_time_days']]
    values = beta.fit(delay_weeks)
    alpha_list.append(values[0])
    beta_list.append(values[1])
    max_list.append(max(delay_weeks))
    min_list.append(min(delay_weeks))
    mean_list.append(mean(delay_weeks))
    
delay_time_df_2 = pd.DataFrame()

delay_time_df_2['Item'] = delay_time_df['Item'].unique()   
delay_time_df_2['alpha_delay'] = alpha_list
delay_time_df_2['beta_delay'] = beta_list
delay_time_df_2['max_delay'] = max_list
delay_time_df_2['min_delay'] = min_list
delay_time_df_2['mean_delay'] = mean_list

In [ ]:
#incoming orders 
incoming_orders_df = \
orders_df_clean[orders_df_clean['EstimatedDeliveryDate']>datetime.now()]
incoming_orders_df = incoming_orders_df[['Date', 'Item', 'PrimaryQty']]
incoming_orders_df = incoming_orders_df.merge(items_df[['ItemTypeID', 
                                                        'ItemType', 'ItemID', 
                                                        'Item']], how = 'left', on = 'Item')

In [ ]:
items_df.columns = ['ItemTypeID', 'ItemType', 
                    'ItemID', 'Item', 'Essentiality', 'Quality', 'DemandPerDay', 'BeginningInventory', 
                   'cost', 'capacity']

In [ ]:
items_df = items_df.merge(delay_time_df_2, how = 'left', on = 'Item')
items_df = items_df[items_df['mean_delay'].notnull()]

In [ ]:
items_df.to_csv('items_df.csv', index=False)
incoming_orders_df.to_csv('incoming_orders_df.csv', index = False)

In [13]:
delay_time_df.to_csv('delay_df.csv', index=False)